# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-14-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-14-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-15 05:26:38,33.93911,67.709953,48718,1969,38252,8497.0,Afghanistan,125.147897,4.041627
1,NaN,NaN,NaN,Albania,2020-12-15 05:26:38,41.15330,20.168300,49191,1016,24820,23355.0,Albania,1709.326569,2.065418
2,NaN,NaN,NaN,Algeria,2020-12-15 05:26:38,28.03390,1.659600,92597,2609,60888,29100.0,Algeria,211.162594,2.817586
3,NaN,NaN,NaN,Andorra,2020-12-15 05:26:38,42.50630,1.521800,7382,79,6706,597.0,Andorra,9554.131884,1.070171
4,NaN,NaN,NaN,Angola,2020-12-15 05:26:38,-11.20270,17.873900,16277,372,8924,6981.0,Angola,49.524941,2.285433


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,47072,47306,47516,47716,47851,48053,48116,48229,48527,48718
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,42148,42988,43683,44436,45188,46061,46863,47742,48530,49191
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,87502,88252,88825,89416,90014,90579,91121,91638,92102,92597


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1864,1874,1900,1906,1919,1935,1945,1956,1965,1969
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,889,905,922,936,951,965,977,989,1003,1016
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2501,2516,2527,2539,2554,2564,2575,2584,2596,2609


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37393,37685,37879,37920,38032,38099,38141,38200,38250,38252
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,21286,21617,22180,22527,23072,23609,24136,24520,24820,24820
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,56617,57146,57648,58146,58146,59135,59590,60028,60457,60888


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2979,3005,3043,3087,3117,3186,3233,3258,3300,3329
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,9626,9728,9821,9974,10087,10288,10489,10665,10806,10898
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1219,1223,1224,1240,1245,1258,1264,1269,1272,1275


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,42,42,42,42,42,42,42,41,41,41
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,138,138,138,141,141,141,141,141,141,141
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,29,29,29,30,30,30,30,30,30,30


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2020-12-15 05:26:38,32.539527,-86.644082,3329,41,0,3288.0,"Autauga, Alabama, US",5958.581682,1.231601
686,1075.0,Lamar,Alabama,US,2020-12-15 05:26:38,33.779950,-88.096680,858,9,0,849.0,"Lamar, Alabama, US",6215.139442,1.048951
687,1077.0,Lauderdale,Alabama,US,2020-12-15 05:26:38,34.901719,-87.656247,5125,61,0,5064.0,"Lauderdale, Alabama, US",5526.857833,1.190244


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,16519462,300479,6399531
India,9906165,143709,9422636
Brazil,6927145,181835,6158049


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,16519462,300479,6399531,9819452,2020-12-15 05:26:38,37.936303,-91.379001
India,9906165,143709,9422636,339820,2020-12-15 05:26:38,23.088275,81.806127
Brazil,6927145,181835,6158049,587261,2020-12-15 05:26:38,-12.669522,-48.480493
Russia,2656601,46846,2105414,504341,2020-12-15 05:26:38,54.546312,62.120860
France,2433859,58391,183340,2192128,2020-12-15 05:26:38,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,1611493,21144,0
Texas,1492917,24440,0
Florida,1134383,20003,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,1611493,21144,0,1590349,2020-12-15 05:26:38,37.843962,-120.728594
Texas,1492917,24440,0,1468477,2020-12-15 05:26:38,31.660643,-98.653069
Florida,1134383,20003,0,1114380,2020-12-15 05:26:38,28.940755,-82.700744
Illinois,856118,15454,0,840664,2020-12-15 05:26:38,39.843618,-89.178330
New York,784204,35427,0,748777,2020-12-15 05:26:38,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,532730,8345,0
Illinois,Cook,355467,7440,0
Arizona,Maricopa,260661,4353,0
Florida,Miami-Dade,260138,4002,0
Texas,Harris,208737,3150,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,532730,8345,0,524385,2020-12-15 05:26:38,34.308284,-118.228241,6037.0
Illinois,Cook,355467,7440,0,348027,2020-12-15 05:26:38,41.841448,-87.816588,17031.0
Arizona,Maricopa,260661,4353,0,256308,2020-12-15 05:26:38,33.348359,-112.491815,4013.0
Florida,Miami-Dade,260138,4002,0,256136,2020-12-15 05:26:38,25.611236,-80.551706,12086.0
Texas,Harris,208737,3150,0,205587,2020-12-15 05:26:38,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,48229,47742,91638,7288,16161,148,1494602,147312,28030,319822,...,1835949,9180,74811,1,107177,1395,108099,2083,18217,11219
2020-12-13,48527,48530,92102,7338,16188,148,1498160,148325,28037,322463,...,1854490,9708,74956,1,107786,1397,109738,2083,18274,11246
2020-12-14,48718,49191,92597,7382,16277,148,1503222,148682,28048,325051,...,1874867,10029,75094,1,108125,1402,111102,2084,18322,11358


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,1956,989,2584,78,366,4,40668,2462,908,4415,...,64123,91,612,0,949,35,961,606,366,307
2020-12-13,1965,1003,2596,79,371,5,40766,2486,908,4473,...,64267,92,612,0,954,35,978,606,367,307
2020-12-14,1969,1016,2609,79,372,5,41041,2503,908,4530,...,64500,95,612,0,960,35,1000,606,367,309


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,38200,24520,60028,6629,8841,138,1330160,124521,25675,277883,...,3805,6155,72084,1,102289,1238,83123,1383,17339,9359
2020-12-13,38250,24820,60457,6629,8898,138,1335317,125169,25676,281106,...,4013,6358,72243,1,102289,1241,85110,1383,17388,9451
2020-12-14,38252,24820,60888,6706,8924,138,1340120,125966,25689,284174,...,4048,6593,72368,1,103037,1246,87057,1384,17409,9554


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2979,3005,3043,3087,3117,3186,3233,3258,3300,3329
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,744,755,759,769,786,806,830,841,855,858
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,4593,4626,4662,4722,4805,4917,4970,5005,5078,5125
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1421,1422,1431,1453,1480,1525,1541,1569,1585,1596
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,8131,8171,8218,8298,8386,8499,8595,8675,8744,8788


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-12,292841,40497,0,402589,184252,1549176,285634,146761,44876,49,...,442953,1472853,231821,5626,1791,278615,199735,62151,465991,38907
2020-12-13,295631,41143,0,408442,185702,1585378,288193,146761,45460,49,...,454305,1481899,233904,5753,1791,281909,202063,63217,468956,39360
2020-12-14,297895,41573,0,420248,187057,1611493,291104,153992,46457,49,...,464624,1492917,235872,5857,1807,285149,203797,64394,471285,39775


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-12        3258   10665    1269  1417   3744     725   1306    7760   
2020-12-13        3300   10806    1272  1441   3776     728   1330    7813   
2020-12-14        3329   10898    1275  1455   3803     728   1340    7872   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-12         1997     1187  ...    1600    381     2293      539   
2020-12-13         2013     1197  ...    1636    390     2323      540   
2020-12-14         2022     1204  ...    1654    392     2345      544   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-12           2384  1886  1289          0      629    428  
2020-12-13           2451  1905  1307          0      639    428  
2020-12-14           2518  1930  1327          0      644    429  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-12,4102,176,0,7322,2911,20958,3871,5363,815,0,...,5400,24308,1038,95,23,4409,2879,966,4324,321
2020-12-13,4102,176,0,7357,2945,21040,3958,5363,816,0,...,5462,24400,1055,95,23,4411,2879,968,4339,321
2020-12-14,4102,176,0,7358,2990,21144,3969,5444,816,0,...,5541,24440,1062,96,23,4414,2918,978,4352,321


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-12          41     141      30   40     47      20     44     129   
2020-12-13          41     141      30   40     47      20     44     129   
2020-12-14          41     141      30   40     47      20     44     129   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-12           57       24  ...       7      8       13        5   
2020-12-13           57       24  ...       7      8       13        5   
2020-12-14           57       24  ...       7      8       13        5   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-12             14     2     7          0       11      2  
2020-12-13             14     2     7          0       11      2  
2020-12-14             14     2     7          0       11      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,0.002348,0.018757,0.005674,0.007186,0.006226,0.006803,0.003541,0.006800,0.000178,0.010238,...,0.011879,0.037405,0.000000,0.0,0.004329,0.002876,0.013853,0.00048,0.003084,0.005107
2020-12-13,0.006179,0.016505,0.005063,0.006861,0.001671,0.000000,0.002381,0.006877,0.000250,0.008258,...,0.010099,0.057516,0.001938,0.0,0.005682,0.001434,0.015162,0.00000,0.003129,0.002407
2020-12-14,0.003936,0.013620,0.005374,0.005996,0.005498,0.000000,0.003379,0.002407,0.000392,0.008026,...,0.010988,0.033066,0.001841,0.0,0.003145,0.003579,0.012430,0.00048,0.002627,0.009959


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,0.005656,0.012282,0.003495,0.000000,0.002740,0.00,0.001527,0.006953,0.0,0.029377,...,0.008176,0.011111,0.0,NaN,0.005297,0.0,0.032223,0.0,0.002740,0.003268
2020-12-13,0.004601,0.014156,0.004644,0.012821,0.013661,0.25,0.002410,0.009748,0.0,0.013137,...,0.002246,0.010989,0.0,NaN,0.005269,0.0,0.017690,0.0,0.002732,0.000000
2020-12-14,0.002036,0.012961,0.005008,0.000000,0.002695,0.00,0.006746,0.006838,0.0,0.012743,...,0.003625,0.032609,0.0,NaN,0.006289,0.0,0.022495,0.0,0.000000,0.006515


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,0.001547,0.015910,0.007350,0.004698,0.004887,0.0,0.004052,0.008480,0.000156,0.016014,...,0.001579,0.036021,0.000000,0.0,0.004577,0.000000,0.024111,0.000000,0.000577,0.003754
2020-12-13,0.001309,0.012235,0.007147,0.000000,0.006447,0.0,0.003877,0.005204,0.000039,0.011598,...,0.054665,0.032981,0.002206,0.0,0.000000,0.002423,0.023904,0.000000,0.002826,0.009830
2020-12-14,0.000052,0.000000,0.007129,0.011616,0.002922,0.0,0.003597,0.006367,0.000506,0.010914,...,0.008722,0.036961,0.001730,0.0,0.007313,0.004029,0.022876,0.000723,0.001208,0.010898


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-12,0.014080,0.013058,NaN,0.020473,0.014469,0.021739,0.014062,0.000000,0.024145,0.0,...,0.015337,0.009208,0.016184,0.015340,0.000000,0.015220,0.021380,0.024968,0.010022,0.003146
2020-12-13,0.009527,0.015952,NaN,0.014538,0.007870,0.023369,0.008959,0.000000,0.013014,0.0,...,0.025628,0.006142,0.008985,0.022574,0.000000,0.011823,0.011655,0.017152,0.006363,0.011643
2020-12-14,0.007658,0.010451,NaN,0.028905,0.007297,0.016472,0.010101,0.049271,0.021931,0.0,...,0.022714,0.007435,0.008414,0.018078,0.008934,0.011493,0.008581,0.018618,0.004966,0.010544


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-12      0.007733  0.016779  0.003956  0.013591  0.022113  0.002766   
2020-12-13      0.012891  0.013221  0.002364  0.016937  0.008547  0.004138   
2020-12-14      0.008788  0.008514  0.002358  0.009715  0.007150  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-12      0.013189  0.013319  0.007568  0.017138  ...  0.003135   
2020-12-13      0.018377  0.006830  0.008012  0.008425  ...  0.022500   
2020-12-14      0.007519  0.007552  0.004471  0.005848  ...  0.011002   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-12      0.007937  0.001310  0.005597   0.003367  0.009636  0.006245   
2020-12-13      0.023622  0.013083  0.001855   0.028104  0.010074  0.013964   
2020-12-14      0.005128  0.009471  0.007407   0.027336  0.013123  0.015302   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-12            NaN  0.014516  0.000000  
2020-12-13            NaN  0.015898  0.000000  
2020-12-14            NaN  0.007825  0.002336  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-12,0.003916,0.113924,NaN,0.010628,0.012522,0.008226,0.006500,0.000000,0.009913,NaN,...,0.013704,0.007794,0.012683,0.021505,0.0,0.008924,0.000000,0.029851,0.013596,0.0
2020-12-13,0.000000,0.000000,NaN,0.004780,0.011680,0.003913,0.022475,0.000000,0.001227,NaN,...,0.011481,0.003785,0.016378,0.000000,0.0,0.000454,0.000000,0.002070,0.003469,0.0
2020-12-14,0.000000,0.000000,NaN,0.000136,0.015280,0.004943,0.002779,0.015103,0.000000,NaN,...,0.014464,0.001639,0.006635,0.010526,0.0,0.000680,0.013546,0.010331,0.002996,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State  Alabama                                                     \
Admin2          Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-12     -0.02381     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-12-13      0.00000     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-12-14      0.00000     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                     ... Wyoming                           \
Admin2          Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-12      0.036364      0.0  ...     0.0    0.0      0.0      0.0   
2020-12-13      0.000000      0.0  ...     0.0    0.0      0.0      0.0   
2020-12-14      0.000000      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-12            0.0   0.0   0.0        NaN      0.0    0.0  
2020-12-13            0.0   0.0   0.0        NaN      0.0    0.0  
2020-12-14            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,0.002473,0.018307,0.005986,0.006362,0.006785,0.005339,0.003950,0.007066,0.000298,0.009626,...,0.011564,0.040010,0.001527,0.0,0.004228,0.003328,0.015542,-0.002965,0.003157,0.006310
2020-12-13,0.004326,0.017406,0.005525,0.006611,0.004228,0.002670,0.003165,0.006971,0.000274,0.008942,...,0.010832,0.048763,0.001733,0.0,0.004955,0.002381,0.015352,-0.001483,0.003143,0.004359
2020-12-14,0.004131,0.015513,0.005450,0.006304,0.004863,0.001335,0.003272,0.004689,0.000333,0.008484,...,0.010910,0.040914,0.001787,0.0,0.004050,0.002980,0.013891,-0.000501,0.002885,0.007159


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,0.005998,0.013160,0.003979,0.000064,0.005478,6.208817e-10,0.003027,0.008884,6.729345e-08,0.027881,...,0.007771,0.012077,0.000410,NaN,0.005530,1.568661e-32,0.027738,-0.001698,0.002095,0.004347
2020-12-13,0.005299,0.013658,0.004312,0.006442,0.009570,1.250000e-01,0.002719,0.009316,3.364672e-08,0.020509,...,0.005008,0.011533,0.000205,NaN,0.005399,7.843305e-33,0.022714,-0.000849,0.002414,0.002173
2020-12-14,0.003668,0.013309,0.004660,0.003221,0.006133,6.250000e-02,0.004732,0.008077,1.682336e-08,0.016626,...,0.004317,0.022071,0.000103,NaN,0.005844,3.921653e-33,0.022604,-0.000425,0.001207,0.004344


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-12,0.001649,0.019132,0.008282,0.008444,0.008661,0.001995,0.004467,0.010286,0.000184,0.015416,...,0.002509,0.034253,0.001544,0.0,0.004497,0.002786,0.025259,-0.003301,0.000944,0.006384
2020-12-13,0.001479,0.015684,0.007714,0.004222,0.007554,0.000997,0.004172,0.007745,0.000112,0.013507,...,0.028587,0.033617,0.001875,0.0,0.002249,0.002605,0.024582,-0.001650,0.001885,0.008107
2020-12-14,0.000766,0.007842,0.007422,0.007919,0.005238,0.000499,0.003884,0.007056,0.000309,0.012211,...,0.018654,0.035289,0.001803,0.0,0.004781,0.003317,0.023729,-0.000464,0.001546,0.009503


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-12,0.014147,0.014607,NaN,0.018460,0.014202,0.022069,0.015231,0.011442,0.021119,8.972083e-83,...,0.015862,0.010110,0.013872,0.019119,0.012446,0.014541,0.018560,0.021841,0.009910,0.007925
2020-12-13,0.011837,0.015279,NaN,0.016499,0.011036,0.022719,0.012095,0.005721,0.017066,4.486042e-83,...,0.020745,0.008126,0.011429,0.020846,0.006223,0.013182,0.015108,0.019496,0.008136,0.009784
2020-12-14,0.009748,0.012865,NaN,0.022702,0.009166,0.019596,0.011098,0.027496,0.019499,2.243021e-83,...,0.021730,0.007780,0.009921,0.019462,0.007578,0.012337,0.011845,0.019057,0.006551,0.010164


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-12      0.011748  0.017309  0.005202  0.018368  0.021686  0.002253   
2020-12-13      0.012320  0.015265  0.003783  0.017652  0.015116  0.003196   
2020-12-14      0.010554  0.011889  0.003071  0.013684  0.011133  0.001598   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-12      0.010231  0.014900  0.006596  0.015400  ...  0.008969   
2020-12-13      0.014304  0.010865  0.007304  0.011912  ...  0.015734   
2020-12-14      0.010911  0.009208  0.005887  0.008880  ...  0.013368   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-12      0.008660  0.006158  0.007904   0.016403  0.009341  0.008759   
2020-12-13      0.016141  0.009621  0.004880   0.022253  0.009708  0.011362   
2020-12-14      0.010635  0.009546  0.006144   0.024795  0.011416  0.013332   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-12           -1.0  0.022450  0.000927  
2020-12-13           -1.0  0.019174  0.000463  
2020-12-14           -1.0  0.013499  0.001400  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-12,0.007944,0.068282,NaN,0.010941,0.014407,0.008604,0.018337,0.003565,0.006249,NaN,...,0.014412,0.008566,0.013701,0.029218,1.259907e-13,0.008841,-0.002894,0.025968,0.014488,0.022046
2020-12-13,0.003972,0.034141,NaN,0.007861,0.013044,0.006258,0.020406,0.001782,0.003738,NaN,...,0.012947,0.006175,0.015039,0.014609,6.299535e-14,0.004647,-0.001447,0.014019,0.008978,0.011023
2020-12-14,0.001986,0.017071,NaN,0.003998,0.014162,0.005601,0.011593,0.008443,0.001869,NaN,...,0.013705,0.003907,0.010837,0.012568,3.149767e-14,0.002664,0.006050,0.012175,0.005987,0.005512


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-12     -0.011904  0.000791  0.001883  0.004173  0.001485  0.000103   
2020-12-13     -0.005952  0.000396  0.000941  0.002087  0.000742  0.000051   
2020-12-14     -0.002976  0.000198  0.000471  0.001043  0.000371  0.000026   

Province_State                                              ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...      Park   
2020-12-12      0.011906  0.002082  0.019474  3.178915e-08  ...  0.002605   
2020-12-13      0.005953  0.001041  0.009737  1.589457e-08  ...  0.001302   
2020-12-14      0.002977  0.000520  0.004868  7.947287e-09  ...  0.000651   

Province_State                                                         \
Admin2            Platte  Sheridan  Sublette Sweetwater         Teton   
2020-12-12      0.000036  0.006726  0.092122   0.032808  9.094947e-13   
2020-12-13      0.000018  0.003363  0.046061   0.016404  4.547474e-13   
2020-12-14      0.000009  0.001682  0.023031   0.008202  2.273737e-13   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2020-12-12      0.013786  -0.501961  0.010157  0.000244  
2020-12-13      0.006893  -0.501961  0.005078  0.000122  
2020-12-14      0.003446  -0.501961  0.002539  0.000061  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201214,AK,40160,NaN,1107400,NaN,totalTestsViral,1147560,140.0,889.0,...,1147560,0,3,58baf833f72d7115b62a7e4dd1ab354526328672,0,0,0,0,0,NaN
1,20201214,AL,297895,53133.0,1478907,NaN,totalTestsPeopleViral,1723669,2286.0,28913.0,...,1776802,0,767,4b16f5bc7ef22ade5ea30159d32917646fba130d,0,0,0,0,0,NaN
2,20201214,AR,187507,26701.0,1705843,NaN,totalTestsViral,1866199,1050.0,9991.0,...,1893350,45,64,9003b761e7be02cd1f88bf8610baaf126a627c3d,0,0,0,0,0,NaN
3,20201214,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,0e77a3838a1af4a4fca74e93d4f31bedd3372edb,0,0,0,0,0,NaN
4,20201214,AZ,420248,15954.0,2126928,NaN,totalTestsViral,4344693,3677.0,31142.0,...,2547176,1,193,f58080d5b42098aad2ef3cc4e3528e7cbfd12245,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201214,AK,40160.0,NaN,1107400.0,NaN,totalTestsViral,1147560.0,140.0,889.0,...,1147560,0,3,58baf833f72d7115b62a7e4dd1ab354526328672,0,0,0,0,0,NaN
1,20201214,AL,297895.0,53133.0,1478907.0,NaN,totalTestsPeopleViral,1723669.0,2286.0,28913.0,...,1776802,0,767,4b16f5bc7ef22ade5ea30159d32917646fba130d,0,0,0,0,0,NaN
2,20201214,AR,187507.0,26701.0,1705843.0,NaN,totalTestsViral,1866199.0,1050.0,9991.0,...,1893350,45,64,9003b761e7be02cd1f88bf8610baaf126a627c3d,0,0,0,0,0,NaN
3,20201214,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,0e77a3838a1af4a4fca74e93d4f31bedd3372edb,0,0,0,0,0,NaN
4,20201214,AZ,420248.0,15954.0,2126928.0,NaN,totalTestsViral,4344693.0,3677.0,31142.0,...,2547176,1,193,f58080d5b42098aad2ef3cc4e3528e7cbfd12245,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-14,AK,40160,NaN,1107400,NaN,totalTestsViral,1147560,140.0,889.0,NaN,...,1147560,0,3,58baf833f72d7115b62a7e4dd1ab354526328672,0,0,0,0,0,NaN
2020-12-14,AL,297895,53133.0,1478907,NaN,totalTestsPeopleViral,1723669,2286.0,28913.0,NaN,...,1776802,0,767,4b16f5bc7ef22ade5ea30159d32917646fba130d,0,0,0,0,0,NaN
2020-12-14,AR,187507,26701.0,1705843,NaN,totalTestsViral,1866199,1050.0,9991.0,372.0,...,1893350,45,64,9003b761e7be02cd1f88bf8610baaf126a627c3d,0,0,0,0,0,NaN
2020-12-14,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,0e77a3838a1af4a4fca74e93d4f31bedd3372edb,0,0,0,0,0,NaN
2020-12-14,AZ,420248,15954.0,2126928,NaN,totalTestsViral,4344693,3677.0,31142.0,829.0,...,2547176,1,193,f58080d5b42098aad2ef3cc4e3528e7cbfd12245,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-14,AK,40160.0,NaN,1107400.0,NaN,totalTestsViral,1147560.0,140.0,889.0,NaN,...,1147560,0,3,58baf833f72d7115b62a7e4dd1ab354526328672,0,0,0,0,0,NaN
2020-12-14,AL,297895.0,53133.0,1478907.0,NaN,totalTestsPeopleViral,1723669.0,2286.0,28913.0,NaN,...,1776802,0,767,4b16f5bc7ef22ade5ea30159d32917646fba130d,0,0,0,0,0,NaN
2020-12-14,AR,187507.0,26701.0,1705843.0,NaN,totalTestsViral,1866199.0,1050.0,9991.0,372.0,...,1893350,45,64,9003b761e7be02cd1f88bf8610baaf126a627c3d,0,0,0,0,0,NaN
2020-12-14,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,0e77a3838a1af4a4fca74e93d4f31bedd3372edb,0,0,0,0,0,NaN
2020-12-14,AZ,420248.0,15954.0,2126928.0,NaN,totalTestsViral,4344693.0,3677.0,31142.0,829.0,...,2547176,1,193,f58080d5b42098aad2ef3cc4e3528e7cbfd12245,0,0,0,0,0,NaN
